In [24]:
import pm4py 

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from sklearn.decomposition import PCA

# Synthetic dataset
from sklearn.datasets import make_classification
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
# Visualization
import matplotlib.pyplot as plt
# Model and performance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report

from pm4py.algo.transformation.log_to_features import algorithm as log_to_features

from helper import create_sample

DICT = {'3-way match, invoice after GR':'DF1', '3-way match, invoice before GR':'DF2', '2-way match':'DF3', 'Consignment':'DF4'}
WINDOWS = False
LOG = '3-way match, invoice after GR'
VERSION = 'DF2'

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
df = create_sample(num_cases=10000, Windows=WINDOWS, version=VERSION)
dataframe = pm4py.format_dataframe(df, case_id='case_concept_name', activity_key='event_concept_name', timestamp_key='event_time_timestamp')
event_log = pm4py.convert_to_event_log(dataframe)
print(event_log)

[{'attributes': {'concept:name': '2000013624_00001'}, 'events': [{'eventID': 189090230173704, 'case_Spend_area_text': 'CAPEX & SOCS', 'case_Company': 'companyID_0000', 'case_Document_Type': 'EC Purchase order', 'case_Sub_spend_area_text': 'MRO (components)', 'case_Purchasing_Document': 2000013624, 'case_Purch._Doc._Category_name': 'Purchase order', 'case_Vendor': 'vendorID_0011', 'case_Item_Type': 'Standard', 'case_Item_Category': '3-way match, invoice before GR', 'case_Spend_classification_text': 'NPR', 'case_Source': 'sourceSystemID_0000', 'case_Name': 'vendor_0011', 'case_GR-Based_Inv._Verif.': False, 'case_Item': 1, 'case_concept_name': '2000013624_00001', 'case_Goods_Receipt': True, 'event_User': 'user_000', 'event_org_resource': 'user_000', 'event_concept_name': 'Record Goods Receipt', 'event_Cumulative_net_worth_(EUR)': 320.0, 'event_time_timestamp': Timestamp('2018-05-22 08:59:00+0000', tz='UTC'), '@@index': 0, '@@case_index': 0, 'log_type': 'DF2', 'concept:name': 'Record Goods

# Earth Mover Distance
The Earth Mover Distance as introduced in:

**Leemans, Sander JJ, Anja F. Syring, and Wil MP van der Aalst. “Earth movers’ stochastic conformance checking.” International Conference on Business Process Management. Springer, Cham, 2019.**


provides a way to calculate the distance between two different stochastic languages.


Generally, one language is extracted from the event log, and one language is extracted from the process model. With language, we mean a set of traces that is weighted according to its probability.


For the event log, trivially taking the set of variants of the log, and dividing by the total number of languages, provides the language of the model.

In [ ]:
language = variant

In [23]:
#data, feature_names = log_to_features.apply(event_log)
data, feature_names = log_to_features.apply(event_log, parameters={"str_tr_attr": [], "str_ev_attr": ["concept:name"], "num_tr_attr": [], "num_ev_attr": ["amount"]})
data, feature_names = log_to_features.apply(event_log, parameters={"str_tr_attr": [], "str_ev_attr": ["concept:name"], "num_tr_attr": [], "num_ev_attr": ["amount"]})

df_features = pd.DataFrame(data, columns=feature_names)
df_features.head(2)

,event:concept:name@Cancel Goods Receipt,event:concept:name@Cancel Invoice Receipt,event:concept:name@Cancel Subsequent Invoice,event:concept:name@Change Approval for Purchase Order,event:concept:name@Change Delivery Indicator,event:concept:name@Change Price,event:concept:name@Change Quantity,event:concept:name@Change Storage Location,event:concept:name@Clear Invoice,event:concept:name@Create Purchase Order Item,event:concept:name@Create Purchase Requisition Item,event:concept:name@Delete Purchase Order Item,event:concept:name@Reactivate Purchase Order Item,event:concept:name@Receive Order Confirmation,event:concept:name@Record Goods Receipt,event:concept:name@Record Invoice Receipt,event:concept:name@Release Purchase Order,event:concept:name@Remove Payment Block,event:concept:name@SRM: Awaiting Approval,event:concept:name@SRM: Change was Transmitted,event:concept:name@SRM: Complete,event:concept:name@SRM: Created,event:concept:name@SRM: Document Completed,event:concept:name@SRM: In Transfer to Execution Syst.,event:concept:name@SRM: Ordered,event:concept:name@Set Payment Block,event:concept:name@Update Order Confirmation,event:concept:name@Vendor creates debit memo,event:concept:name@Vendor creates invoice,event:org:resource@UNDEFINED,succession:concept:name@Cancel Goods Receipt#Cancel Invoice Receipt,succession:concept:name@Cancel Goods Receipt#Vendor creates debit memo,succession:concept:name@Cancel Invoice Receipt#Cancel Goods Receipt,succession:concept:name@Cancel Invoice Receipt#Clear Invoice,succession:concept:name@Cancel Invoice Receipt#Create Purchase Order Item,succession:concept:name@Cancel Invoice Receipt#Record Invoice Receipt,succession:concept:name@Cancel Invoice Receipt#Remove Payment Block,succession:concept:name@Cancel Invoice Receipt#Vendor creates invoice,succession:concept:name@Cancel Subsequent Invoice#Clear Invoice,succession:concept:name@Change Approval for Purchase Order#Change Approval for Purchase Order,succession:concept:name@Change Approval for Purchase Order#Change Price,succession:concept:name@Change Approval for Purchase Order#Change Quantity,succession:concept:name@Change Approval for Purchase Order#Create Purchase Order Item,succession:concept:name@Change Approval for Purchase Order#Receive Order Confirmation,succession:concept:name@Change Approval for Purchase Order#Release Purchase Order,succession:concept:name@Change Approval for Purchase Order#Vendor creates invoice,succession:concept:name@Change Delivery Indicator#Change Delivery Indicator,succession:concept:name@Change Delivery Indicator#Change Quantity,succession:concept:name@Change Delivery Indicator#Record Goods Receipt,succession:concept:name@Change Delivery Indicator#Record Invoice Receipt,succession:concept:name@Change Delivery Indicator#Vendor creates debit memo,succession:concept:name@Change Delivery Indicator#Vendor creates invoice,succession:concept:name@Change Price#Change Approval for Purchase Order,succession:concept:name@Change Price#Change Price,succession:concept:name@Change Price#Change Quantity,succession:concept:name@Change Price#Clear Invoice,succession:concept:name@Change Price#Create Purchase Order Item,succession:concept:name@Change Price#Create Purchase Requisition Item,succession:concept:name@Change Price#Receive Order Confirmation,succession:concept:name@Change Price#Record Goods Receipt,succession:concept:name@Change Price#Record Invoice Receipt,succession:concept:name@Change Price#Release Purchase Order,succession:concept:name@Change Price#Remove Payment Block,succession:concept:name@Change Price#Vendor creates invoice,succession:concept:name@Change Quantity#Change Delivery Indicator,succession:concept:name@Change Quantity#Change Price,succession:concept:name@Change Quantity#Change Quantity,succession:concept:name@Change Quantity#Clear Invoice,succession:concept:name@Change Quantity#Create Purchase Order Item,succession:concept:name@Change Quantity#Create Purchase Requisition Item,succession:concept:name@Chang

In [27]:
pca = PCA(n_components=5)
df2 = pd.DataFrame(pca.fit_transform(df_features))
model=IsolationForest()
model.fit(df2)
df2["scores"] = model.decision_function(df2)
df2["@@index"] = df2.index
df2 = df2[["scores", "@@index"]]
df2 = df2.sort_values("scores")
print(df2)

       scores  @@index
810 -0.125738      810
865 -0.119217      865
772 -0.119217      772
835 -0.114946      835
755 -0.114269      755
775 -0.113142      775
763 -0.113142      763
665 -0.110003      665
863 -0.110003      863
847 -0.110003      847
626 -0.110003      626
742 -0.108687      742
687 -0.105958      687
139 -0.098984      139
721 -0.098829      721
484 -0.095665      484
794 -0.093897      794
662 -0.093479      662
682 -0.093479      682
61  -0.093301       61
678 -0.089824      678
840 -0.083153      840
658 -0.082794      658
263 -0.082743      263
660 -0.082593      660
748 -0.081828      748
174 -0.081631      174
710 -0.079263      710
789 -0.078675      789
715 -0.077977      715
869 -0.075034      869
736 -0.075034      736
822 -0.072854      822
686 -0.070919      686
653 -0.069415      653
799 -0.069415      799
807 -0.069415      807
793 -0.069415      793
771 -0.069415      771
809 -0.069415      809
674 -0.069415      674
34  -0.069415       34
732 -0.0694